In [ ]:
# Install packages from requirements.txt
!pip install -r requirements.txt

In [ ]:
from PIL import Image, ImageDraw
from PIL import ImageFont
import random
import os
import numpy as np

def get_shuffled_file_paths(directory):
    file_paths = []

    # Walk through the directory and collect all file paths
    for root, dirs, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            if os.path.isfile(file_path):
                file_paths.append(file_path)

    # Shuffle the list of file paths
    random.shuffle(file_paths)

    return file_paths


def mask_random(image_path, output_path = "./diffuser_images"): 
    # Open the image
    image = Image.open(image_path)
    width, height = image.size

    # Create a new black or white image, depending on the random choice
    new_image = Image.new("RGB", (width, height))
    draw = ImageDraw.Draw(new_image)

    choice = random.choice(["upper_white", "upper_black", "left_white", "left_black", "random_patch"])
    if choice == "random_patch":
        percentage=random.randint(40, 60)
        new_image = Image.new("RGB", (width, height), color=(0, 0, 0))
        draw = ImageDraw.Draw(new_image)
        total_white_area = int((width * height) * (percentage / 100))
        white_area = 0
        while white_area < total_white_area:
            # Generate random position and size for the white patch
            x, y = random.randint(0, width - 1), random.randint(0, height - 1)
            w, h = random.randint(width // 16, width // 4+1), random.randint(height // 16, height // 4+1)        
            # Draw the white patch
            draw.rectangle([x, y, x + w, y + h], fill=(255, 255, 255))   
            # Update the current area of white patches
            white_area = sum([1 for pixel in new_image.getdata() if pixel == (255, 255, 255)])
    elif choice == "upper_white":
        draw.polygon([(0, 0), (width, 0), (width, height)], fill=(255, 255, 255))
        draw.polygon([(0, 0), (0, height), (width, height)], fill=(0, 0, 0))
    elif choice == "upper_black":
        draw.polygon([(0, 0), (width, 0), (width, height)], fill=(0, 0, 0))
        draw.polygon([(0, 0), (0, height), (width, height)], fill=(255, 255, 255))
    elif choice == "left_white":
        draw.polygon([(0, 0), (width, 0), (0, height)], fill=(0, 0, 0))
        draw.polygon([(0, height), (width, height), (width, 0)], fill=(255, 255, 255))
    elif choice == "left_black":
        draw.polygon([(0, 0), (width, 0), (0, height)], fill=(255, 255, 255))
        draw.polygon([(0, height), (width, height), (width, 0)], fill=(0, 0, 0))
    print(choice)
        
    # Save the masked image
    image_path = os.path.join(output_path, "mask.png")
    new_image.save(image_path)

    return new_image


def mask_top_half(image_path, output_path = "./diffuser_images"):
    # Open the image 
    image = Image.open(image_path)

    # Create a new image with the same size as the input image
    width, height = image.size
    masked_image = Image.new('RGB', (width, height))

    # Draw a black rectangle on the top half and a white rectangle on the bottom half
    draw = ImageDraw.Draw(masked_image)
    black_color = (0, 0, 0)  # Black
    white_color = (255, 255, 255)  # White
    draw.rectangle([(0, 0), (width, height // 2)], fill=black_color)
    draw.rectangle([(0, height // 2), (width, height)], fill=white_color)

    # Save the masked image
    image_path = os.path.join(output_path, "mask2.png")
    masked_image.save(image_path)

    # Return the PIL image object
    return masked_image

def compose_side_by_side(left_image, right_image, output_path, margin=10):
    left_label = "inpainting"
    right_label = "original"
    width, height = left_image.size
    new_width = width * 2 + margin
    new_height = height + 40
    font = ImageFont.truetype("arial.ttf", size=30)

    # Create a new image with the combined dimensions
    # Calculate the dimensions of the new image
    width, height = left_image.size
    new_width = width * 2 + margin
    hiegt_increase = 40
    new_height = height + hiegt_increase

    # Create a new image with the combined dimensions
    combined_image = Image.new("RGB", (new_width, new_height), color=(255, 255, 255))

    # Paste the two input images side by side with a margin
    combined_image.paste(left_image, (0, 0))
    combined_image.paste(right_image, (width + margin, 0))

    # Add text labels below the images
    draw = ImageDraw.Draw(combined_image)
    left_text_bbox = draw.textbbox((0, 0), left_label, font=font)
    left_text_width, left_text_height = left_text_bbox[2] - left_text_bbox[0], left_text_bbox[3] - left_text_bbox[1]
    right_text_bbox = draw.textbbox((0, 0), right_label, font=font)
    right_text_width, right_text_height = right_text_bbox[2] - right_text_bbox[0], right_text_bbox[3] - right_text_bbox[1]
    draw.text(((width - left_text_width) // 2, height), left_label, font=font, fill=(0, 0, 0))
    draw.text((width//2 +(width * 2 + margin - right_text_width/2) // 2, height), right_label, font=font, fill=(0, 0, 0))

    # Save the composite image
    image_path = os.path.join(output_path, "group.png")
    combined_image.save(image_path)

    # Return the PIL image object
    return combined_image





In [ ]:
import torch
from PIL import Image
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from tools import mask_top_half, compose_side_by_side, get_shuffled_file_paths, mask_random
from diffusers import StableDiffusionInpaintPipeline

# Initialize the pipeline
pipe = StableDiffusionInpaintPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-inpainting", torch_dtype=torch.float32
)
pipe.to("cpu")

# Define the dataset directory
directory = "./similar/inpainting" # edit path
list_files = get_shuffled_file_paths(directory)

# Process the images
for image_address in list_files:
    print(image_address)
    
    # Only process PNG images
    if not image_address.lower().endswith(".png"):
        continue
    
    # Open the image and extract picture number (folder name before the file name)
    image = Image.open(image_address).convert("RGB")
    width, height = image.size
    picture_number = os.path.basename(os.path.dirname(image_address))
    
    # Create directory based on the picture number
    output_path = f"./diffuser_images/{picture_number}"
    os.makedirs(output_path, exist_ok=True)
    
    # Apply random masking
    masked_image = mask_random(image_address, output_path=f"{output_path}/mask.png").convert("RGB").resize((128, 128))
    
    # Inpainting prompt
    prompt = "generate a painting compatible with the rest of the image"
    
    # Perform inpainting
    image_inpainting = pipe(prompt=prompt, image=image.resize((128, 128)), mask_image=masked_image).images[0]
    
    # Compose and save the images
    compose_side_by_side(image_inpainting.resize((width, height)), image.resize((128, 128)).resize((width, height)), f"{output_path}/group.png")
    image_inpainting.resize((width, height)).save(f"{output_path}/inpainting.png")
    image.resize((128, 128)).resize((width, height)).save(f"{output_path}/original.png")

